# Prueba de la extración de feature vectors

In [10]:
#Importación de librerías
import numpy as np
import pandas as pd
from mne import io
from FVfunctions import getMeData, getMeFeatures
from features_stats import stats_features
from dataset_reader import get_seizure_events, get_seizure_array

In [2]:
%load_ext autoreload
%autoreload 2

#### 1. Lectura

In [3]:
DATA_DIR = r"../eeg_dataset/physionet.org/files/siena-scalp-eeg/1.0.0/"
path01 = rf"{DATA_DIR}PN05/PN05-2.edf"
raw01 = io.read_raw_edf(path01)
paciente = "PN05"
realizacion = "2"

Extracting EDF parameters from C:\Users\gonza\Documents\ITBA\2024 1C\PSIB\TP FINAL PSIB\eeg_dataset\physionet.org\files\siena-scalp-eeg\1.0.0\PN05\PN05-2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\gonza\AppData\Local\Temp\ipykernel_22080\2785477139.py:3: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw01 = io.read_raw_edf(path01)
C:\Users\gonza\AppData\Local\Temp\ipykernel_22080\2785477139.py:3: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw01 = io.read_raw_edf(path01)
C:\Users\gonza\AppData\Local\Temp\ipykernel_22080\2785477139.py:3: RuntimeWarning: Highpass cutoff frequency 15.91549 is greater than lowpass cutoff frequency 15.0, setting values to 0 and Nyquist.
  raw01 = io.read_raw_edf(path01)


#### 2. Info del archivo

In [4]:
info = raw01.info
print(info)
print(info['ch_names'])

<Info | 8 non-empty values
 bads: []
 ch_names: EEG Fp1, EEG F3, EEG C3, EEG P3, EEG O1, EEG F7, EEG T3, EEG T5, ...
 chs: 37 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 256.0 Hz
 meas_date: 2016-01-01 06:46:02 UTC
 nchan: 37
 projs: []
 sfreq: 512.0 Hz
 subject_info: 1 item (dict)
>
['EEG Fp1', 'EEG F3', 'EEG C3', 'EEG P3', 'EEG O1', 'EEG F7', 'EEG T3', 'EEG T5', 'EEG Fc1', 'EEG Fc5', 'EEG Cp1', 'EEG Cp5', 'EEG F9', 'EEG Fz', 'EEG Cz', 'EEG Pz', 'EEG F4', 'EEG C4', 'EEG P4', 'EEG O2', 'EEG F8', 'EEG T4', 'EEG T6', 'EEG Fc2', 'EEG Fc6', 'EEG Cp2', 'EEG Cp6', 'EEG F10', 'EKG EKG', 'SPO2', 'HR', '1', '2', 'EEG FP2', 'EEG P9', 'EEG P10', 'MK']


#### 3. Obtención de las mediciones a utilizar y otros datos importantes

In [5]:
# Se obtiene el nombre de todos los canales
ch_nms = info["ch_names"]

# Se obtienen los canales seleccionados del lazo izquierdo
filt_ch_nms = ['EEG T3','EEG T5','EEG F7','EEG F3','EEG C3','EEG P3']

# Seleccionar los datos de los canales filtrados
raw01_filt = raw01.pick(filt_ch_nms)

# Obtener los datos de los canales filtrados por nombre
data_namefilt = raw01_filt.get_data()

# Se convierten las mediciones a microvoltios
data_namefilt = data_namefilt * 1e6
data_namefilt_df = pd.DataFrame(data=data_namefilt.T, columns=filt_ch_nms)

# Dimensiones de 'data_filt'
dim_data_filt = np.shape(data_namefilt)

# Verifición
print("Canales filtrados:", filt_ch_nms)
print(f"Cantidad de canales resultantes: {dim_data_filt[0]}")
print(f"Cantidad de datos en cada canal: {dim_data_filt[1]}")
data_namefilt_df.head(10)

Canales filtrados: ['EEG T3', 'EEG T5', 'EEG F7', 'EEG F3', 'EEG C3', 'EEG P3']
Cantidad de canales resultantes: 6
Cantidad de datos en cada canal: 4733440


,EEG T3,EEG T5,EEG F7,EEG F3,EEG C3,EEG P3
0,-15.000,16.625,-5.000,2.500,-5.125,-16.500
1,-11.125,30.125,-3.250,6.750,-3.500,-4.000
2,-6.750,24.125,-0.875,-1.750,-1.125,4.000
3,-2.375,12.750,0.500,-8.250,-1.500,4.500
4,3.000,9.250,2.875,-4.500,-2.625,3.250
5,5.500,18.875,2.750,-19.500,-1.750,0.375
6,2.250,19.750,1.250,-27.875,-1.125,-6.000
7,0.125,5.125,2.750,-4.000,-1.750,-12.000
8,2.875,5.000,3.625,3.500,-0.125,-14.125
9,2.500,16.625,1.125,-13.625,2.125,-16.625


In [6]:
# Frecuencia de muestreo y cantidad de muestras tomadas
fs = info["sfreq"]
len_data = dim_data_filt[1]

# Array de instantes
start = 0
stop = (1 / fs) * len_data
arr_t = np.arange(start=start, stop=stop, step=(1 / fs))

# Verificación de datos
print("Frecuencia de muestreo:", fs)
print("Instantes [s]:", arr_t[0], "...", arr_t[-1])

Frecuencia de muestreo: 512.0
Instantes [s]: 0.0 ... 9244.998046875


#### 4. Generacion de segmentos y features

In [8]:
# Encontramos las seizures del archivo
path_seiz = rf"{DATA_DIR}PN05/Seizures-list-PN05.txt"
seizures = get_seizure_events(path_seiz)
seizures_in_02 = [seiz for seiz in seizures if seiz["file_name"] == "PN05-2.edf"]
arr_mtx_t_epi = get_seizure_array(seizures_in_02)
seizures_in_02

[{'seizure_number': 2,
  'file_name': 'PN05-2.edf',
  'registration_start_time': '06.46.02',
  'registration_end_time': '09.19.47',
  'seizure_start_time': '08.45.25',
  'seizure_end_time': '08.46.00'}]

In [9]:
# Matriz de inicio de registro y final de registro
seiz = seizures_in_02[0] # una sola !!
mtx_t_reg = np.array([seiz['registration_start_time'], seiz['registration_end_time']])

# Utilización de la función para una única señal
list_segs, arr_labels = getMeData(data_namefilt, mtx_t_reg=mtx_t_reg, arr_mtx_t_epi=arr_mtx_t_epi, winlen=2, proportion=0.4)
list_segs[0]

array([[ -6.44861197, -14.41007862, -21.80983335, ...,  -4.48831135,
         -4.21901134,  -2.8510888 ],
       [ -5.88095295,  -3.27032485,  -3.43263646, ...,  12.59162582,
          7.36264985,   1.6076962 ],
       [ -0.83301453,  -1.25101932,  -2.04141079, ..., -23.45783771,
        -24.17600637, -25.67897428],
       ...,
       [  0.68034242,  -4.00761189,  -8.34748549, ...,  -5.61545513,
         -5.87480518,  -5.25658974],
       [  0.92877445,  -0.44348798,  -4.31283595, ..., -23.95445503,
        -16.12186009, -10.22864701],
       [  0.90055912, -11.03868087, -14.03135442, ...,  11.33054779,
         10.28751314,   7.81128204]])

In [11]:
# Potencia
fs = info["sfreq"]
df_fv = getMeFeatures(list_segs, filt_ch_nms, fs)
df_fv

,EEG T3_potRelDelta,EEG T3_potRelTheta,EEG T3_potRelAlpha,EEG T3_potRelBeta,EEG T3_potRelGamma,EEG T5_potRelDelta,EEG T5_potRelTheta,EEG T5_potRelAlpha,EEG T5_potRelBeta,EEG T5_potRelGamma,...,EEG C3_RMS,EEG C3_skewness,EEG C3_media,EEG C3_devest,EEG P3_kurtosis,EEG P3_RMS,EEG P3_skewness,EEG P3_media,EEG P3_devest,coordination
0,217.644236,49.269005,89.263621,95.580341,55.646721,124.140570,58.149563,108.482934,212.933265,137.980287,...,11.677248,-0.231875,-0.502056,11.666451,4.401967,79.218336,-1.720705,-11.207909,78.421474,0.742638
1,178.283112,68.107614,68.265979,55.249263,21.184491,32.760069,111.496744,28.529220,60.629094,50.598366,...,8.416224,-0.156994,-0.196605,8.413928,-0.094271,10.886250,0.124006,7.289688,8.085227,0.414776
2,148.265309,81.879942,29.160342,32.561074,17.223644,24.356829,170.406820,28.689113,27.480016,35.805629,...,8.980941,0.048354,-0.762440,8.948519,0.274183,8.071810,0.386512,0.553549,8.052807,0.432389
3,131.956727,91.343940,18.468434,41.782452,16.610201,63.404226,37.728971,35.714350,72.280008,92.015080,...,8.142055,0.173270,-0.372658,8.133523,0.824657,8.899896,-0.250272,0.663754,8.875110,0.418980
4,171.847087,80.592865,21.834394,24.324345,9.639249,19.700832,17.899795,19.355433,53.485841,56.392681,...,8.322849,0.168588,-0.297015,8.317548,-0.508299,7.467841,0.136701,3.939685,6.344093,0.517779
5,49.951440,266.194512,118.488910,33.968660,10.059972,19.399979,148.065837,32.526176,45.725322,35.489166,...,10.455694,-0.372698,0.004679,10.455693,-0.086786,7.306819,0.158408,1.648406,7.118452,0.689118
6,60.522752,694.580842,171.390084,42.324166,8.917114,105.642761,176.668021,87.696803,60.246497,43.777468,...,13.968668,-0.294790,-1.210592,13.916111,0.120155,8.249094,-0.085026,-1.848924,8.039218,0.630907
7,46.869211,908.317318,155.500322,71.472523,11.021553,36.499778,173.734693,46.672212,42.406688,26.710877,...,15.193188,-0.095495,-0.903271,15.166313,-0.533101,8.779397,0.032263,-2.878586,8.294068,0.709410
8,76.570210,566.605445,24.871353,73.751879,16.436683,31.935764,116.049523,54.677429,71.570142,40.512719,...,13.386318,-0.305510,0.428950,13.379443,-0.135368,8.074814,0.008525,0.413674,8.064211,0.525505
9,80.895259,92.963097,126.074663,172.348406,22.912454,75.217608,230.474477,39.743363,197.952283,29.865994,...,15.989466,-1.146662,-2.129462,15.847032,5.242932,22.302080,-1.793662,-5.856859,21.519294,0.445882
